<a href="https://colab.research.google.com/github/jackma-00/peft-of-a-llm/blob/main/lora_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>